# Combining scheduled and actual headway calcs
### Export a geojson for a given route and day, containing all bus stops on that route with headway stats for each bus stop on that route and day

In [3]:
# import chn-ghost-buses files
import sys
import geopandas as gpd
sys.path.append('/Users/kristenhahn/repos/chn-ghost-buses')
from data_analysis.static_gtfs_analysis import *
# from scrape_data.scrape_data import *
from scrape_data.scrape_schedule_versions import *

# import headways file
from headways import *

In [4]:
# Values to use for testing
gtfs_version_id = '20230721'

In [5]:
# Use Laurie's code to get gtfs feed data
gtfs_feed = download_extract_format(gtfs_version_id) 

INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230721
Loading shapes.txt: 100%|██████████| 7/7 [00:04<00:00,  1.58it/s]


# Get headway stats for an entire route on a single day
Takes about 10 minutes per route, depending on size of the route, on my computer

## Route 124 on 7/26/2023

In [6]:
%%capture --no-display


# Smaller route for testing: Route 124, Navy Pier - 7/26/2023

stats_all_stops_124_20230726 = get_stats_all_stops(gtfs_feed, '124', '2023-07-26')

INFO:pyogrio._io:Created 24 records
INFO:pyogrio._io:Created 2 records


In [7]:
stats_all_stops_124_20230726

,stop id,route_id,date,day,direction,Actual mean headway (minutes),Actual 25th percentile headway (minutes),Actual median headway (minutes),Actual 75th percentile headway (minutes),Scheduled mean headway (minutes),Scheduled 25th percentile headway (minutes),Scheduled median headway (minutes),Scheduled 75th percentile headway (minutes),stop name,geometry
0,5509,124,2023-07-26,Wednesday,Eastbound,17,13,17,20,17,14,15,21,Columbus (Mid) & E Wacker,POINT (-87.62055 41.88726)
1,5514,124,2023-07-26,Wednesday,Westbound,17,14,16,20,17,14,16,20,Columbus & North Water,POINT (-87.62051 41.88973)
2,18122,124,2023-07-26,Wednesday,Eastbound,17,14,16,20,17,14,15,21,Washington & Clark/Dearborn,POINT (-87.63048 41.88318)
3,446,124,2023-07-26,Wednesday,Eastbound,17,13,16,20,17,14,15,21,Washington & Lasalle,POINT (-87.63282 41.88318)
4,5027,124,2023-07-26,Wednesday,Eastbound,18,14,17,21,17,14,15,22,Canal & Madison,POINT (-87.63959 41.88206)
5,450,124,2023-07-26,Wednesday,Westbound,17,13,16,19,17,14,16,20,Madison & Wabash,POINT (-87.62591 41.88214)
6,18126,124,2023-07-26,Wednesday,Westbound,17,13,17,20,17,14,16,20,Madison & Dearborn/State,POINT (-87.62910 41.88202)
7,5006,124,2023-07-26,Wednesday,Westbound,17,12,17,19,17,14,16,19,Madison & Canal,POINT (-87.63903 41.88192)
8,5510,124,2023-07-26,Wednesday,Eastbound,17,13,17,20,17,14,15,21,Columbus & North Water,POINT (-87.62028 41.88943)
9,754,124,2023-07-26,Wednesday,Eastbound,19,13,17,22,17,14,15,21,Illinois & McClurg,POINT (-87.61785 41.89102)


In [8]:
# plot route 124 linestring and stops with headway data

linestring_124 = gpd.read_file('headway_summaries/route124_linestring.json')
stops_124 = gpd.read_file('headway_summaries/route124_2023-07-26.json')

m = linestring_124.explore(color='#41B6E6', tiles="CartoDB positron")
stats_all_stops_124_20230726.explore(m=m, color='#E4002B', marker_kwds=({'radius':3.0}))

########### Viewing the stops json isn't working.  Investigate this further.#############
#stops_124.explore(m=m, color='#E4002B', marker_kwds=({'radius':3.0}))


## Route 74 on 7/26/2023

In [9]:
%%capture --no-display

# Route 74, 7/26/2023

stats_all_stops_74_20230726 = get_stats_all_stops(gtfs_feed, '74', '2023-07-26')

INFO:pyogrio._io:Created 300 records
INFO:pyogrio._io:Created 6 records


In [10]:
stats_all_stops_74_20230726

####################
### Bus stops are each showing 3 times. Maybe once per pattern?  
### To be investigated.   Route 124 did not display this duplication.
#####################

,stop id,route_id,date,day,direction,Actual mean headway (minutes),Actual 25th percentile headway (minutes),Actual median headway (minutes),Actual 75th percentile headway (minutes),Scheduled mean headway (minutes),Scheduled 25th percentile headway (minutes),Scheduled median headway (minutes),Scheduled 75th percentile headway (minutes),stop name,geometry
0,1330,74,2023-07-26,Wednesday,Eastbound,15,10,14,20,14,9,13,20,Fullerton & Washtenaw,POINT (-87.69529 41.92477)
1,1330,74,2023-07-26,Wednesday,Eastbound,15,10,14,20,14,9,13,20,Fullerton & Washtenaw,POINT (-87.69529 41.92477)
2,15986,74,2023-07-26,Wednesday,Westbound,15,9,14,21,15,9,14,20,Fullerton & Lavergne,POINT (-87.75162 41.92428)
3,15986,74,2023-07-26,Wednesday,Westbound,15,9,14,21,15,9,14,20,Fullerton & Lavergne,POINT (-87.75162 41.92428)
4,15986,74,2023-07-26,Wednesday,Westbound,15,9,14,21,15,9,14,20,Fullerton & Lavergne,POINT (-87.75162 41.92428)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1297,74,2023-07-26,Wednesday,Eastbound,15,9,16,20,14,9,14,20,Fullerton & Major,POINT (-87.76857 41.92387)
296,1297,74,2023-07-26,Wednesday,Eastbound,15,9,16,20,14,9,14,20,Fullerton & Major,POINT (-87.76857 41.92387)
297,1297,74,2023-07-26,Wednesday,Eastbound,15,9,16,20,14,9,14,20,Fullerton & Major,POINT (-87.76857 41.92387)
298,1228,74,2023-07-26,Wednesday,Westbound,16,9,15,20,14,9,12,20,Fullerton & Surrey Court,POINT (-87.66027 41.92529)


In [11]:
# plot route 74 linestring and stops with headway data

linestring_74 = gpd.read_file('headway_summaries/route74_linestring.json')
stops_74 = gpd.read_file('headway_summaries/route74_2023-07-26.json')

m = linestring_74.explore(color='#41B6E6', tiles="CartoDB positron")
stats_all_stops_74_20230726.explore(m=m, color='#E4002B', marker_kwds=({'radius':3.0}))

In [12]:
%%capture --no-display

# # Route 55, 7/24/2023

stats_all_stops_55_20230724 = get_stats_all_stops(gtfs_feed, '55', '2023-07-24')


INFO:pyogrio._io:Created 346 records
INFO:pyogrio._io:Created 8 records


In [13]:
stats_all_stops_55_20230724

####################
### Bus stops are each showing 4 times. Maybe once per pattern?  
### To be investigated.   Route 124 did not display this duplication. Route 74 showed each stop 3 times.
#####################


,stop id,route_id,date,day,direction,Actual mean headway (minutes),Actual 25th percentile headway (minutes),Actual median headway (minutes),Actual 75th percentile headway (minutes),Scheduled mean headway (minutes),Scheduled 25th percentile headway (minutes),Scheduled median headway (minutes),Scheduled 75th percentile headway (minutes),stop name,geometry
0,16096,55,2023-07-24,Monday,Eastbound,17,11,17,21,16,12,15,20,Garfield & Lasalle,POINT (-87.62956 41.79419)
1,16096,55,2023-07-24,Monday,Eastbound,17,11,17,21,16,12,15,20,Garfield & Lasalle,POINT (-87.62956 41.79419)
2,18018,55,2023-07-24,Monday,Eastbound,17,11,16,21,16,12,15,20,55th Street & University,POINT (-87.59768 41.79501)
3,18018,55,2023-07-24,Monday,Eastbound,17,11,16,21,16,12,15,20,55th Street & University,POINT (-87.59768 41.79501)
4,10600,55,2023-07-24,Monday,Westbound,17,12,16,21,16,11,14,19,Garfield & Throop,POINT (-87.65771 41.79415)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,17022,55,2023-07-24,Monday,Westbound,15,9,14,21,13,10,12,17,55th Street & Spaulding,POINT (-87.70587 41.79354)
342,17022,55,2023-07-24,Monday,Westbound,15,9,14,21,13,10,12,17,55th Street & Spaulding,POINT (-87.70587 41.79354)
343,17022,55,2023-07-24,Monday,Westbound,15,9,14,21,13,10,12,17,55th Street & Spaulding,POINT (-87.70587 41.79354)
344,10506,55,2023-07-24,Monday,Eastbound,16,11,15,20,14,10,13,17,3600 W 55th Street,POINT (-87.71358 41.79330)


In [14]:
# plot route 55 linestring and stops with headway data

linestring_55 = gpd.read_file('headway_summaries/route55_linestring.json')
stops_55 = gpd.read_file('headway_summaries/route55_2023-07-26.json')

m = linestring_55.explore(color='#41B6E6', tiles="CartoDB positron")
stats_all_stops_55_20230724.explore(m=m, color='#E4002B', marker_kwds=({'radius':3.0}))

# Get detailed arrival times and headways all day for a single stop and route on a single day

## 7/25/2023, Route 8 (Halsted), stop 5864 Southbound

In [15]:

direction = 'Southbound'
stop_id = '5864'
service_date_string = '2023-07-25'
route_id = '8'


In [16]:
vehicles = get_chn_vehicles(service_date_string)
vehicles

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,tatripid,origtatripno,tablockid,zone,scrape_file,data_time,data_hour,data_date
0,7949,2023-07-25 00:02:00+00:00,41.894199,-87.620270,116,18414,3,Michigan/Chicago,70716,False,398,245955419,3 -707,NaN,bus_data/2023-07-25/00:02:56.json,2023-07-25 00:02:00,0,2023-07-25
1,7989,2023-07-25 00:02:00+00:00,41.893967,-87.619885,178,18414,3,Michigan/Chicago,70888,True,397,245955416,3 -711,NaN,bus_data/2023-07-25/00:02:56.json,2023-07-25 00:02:00,0,2023-07-25
2,1262,2023-07-25 00:02:00+00:00,41.871794,-87.624252,177,18415,3,95th/RED LINE,11645,False,1080331,245955775,3 -717,NaN,bus_data/2023-07-25/00:02:56.json,2023-07-25 00:02:00,0,2023-07-25
3,1314,2023-07-25 00:02:00+00:00,41.819835,-87.616943,178,18415,3,95th/RED LINE,32001,False,1080330,245955591,3 -708,NaN,bus_data/2023-07-25/00:02:56.json,2023-07-25 00:02:00,0,2023-07-25
4,7926,2023-07-25 00:02:00+00:00,41.776231,-87.615639,178,18415,3,95th/RED LINE,47949,False,1080329,245954565,N4 -793,NaN,bus_data/2023-07-25/00:02:56.json,2023-07-25 00:02:00,0,2023-07-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163572,4335,2023-07-26 23:57:00+00:00,41.981577,-87.655151,178,8102,151,Union Station,10818,False,403,245945780,151 -513,NaN,bus_data/2023-07-26/23:57:56.json,2023-07-26 23:57:00,23,2023-07-26
163573,4378,2023-07-26 23:57:00+00:00,41.948733,-87.643506,152,8102,151,Union Station,25619,False,398,245945719,151 -518,NaN,bus_data/2023-07-26/23:57:56.json,2023-07-26 23:57:00,23,2023-07-26
163574,4133,2023-07-26 23:57:00+00:00,41.893646,-87.624306,177,8102,151,Union Station,47990,False,400,245945709,151 -516,NaN,bus_data/2023-07-26/23:57:56.json,2023-07-26 23:57:00,23,2023-07-26
163575,1824,2023-07-26 23:57:00+00:00,42.001766,-87.660767,178,904,155,Kedzie,3344,False,1005709,245945883,155 -552,NaN,bus_data/2023-07-26/23:57:56.json,2023-07-26 23:57:00,23,2023-07-26


In [17]:
stop_details = get_scheduled_stop_details(gtfs_feed, route_id, service_date_string)
stop_details

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,arrival_hour,departure_hour,route_id,service_id,direction,raw_date,stop_time
0,6570000030020,10:38:00,10:38:00,3681,1,Waveland/Broadway,0,0,10,10,8,65701,Northbound,2023-07-25 00:00:00+00:00,2023-07-25 10:38:00+00:00
3,6570000030020,10:38:42,10:38:42,17510,2,Waveland/Broadway,0,555,10,10,8,65701,Northbound,2023-07-25 00:00:00+00:00,2023-07-25 10:38:42+00:00
6,6570000030020,10:39:29,10:39:29,5866,3,Waveland/Broadway,0,1048,10,10,8,65701,Northbound,2023-07-25 00:00:00+00:00,2023-07-25 10:39:29+00:00
9,6570000030020,10:40:28,10:40:28,15085,4,Waveland/Broadway,0,1877,10,10,8,65701,Northbound,2023-07-25 00:00:00+00:00,2023-07-25 10:40:28+00:00
12,6570000030020,10:41:21,10:41:21,17231,5,Waveland/Broadway,0,2511,10,10,8,65701,Northbound,2023-07-25 00:00:00+00:00,2023-07-25 10:41:21+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45827,6570044906020,06:40:58,06:40:58,5968,99,Waveland/Broadway,0,72297,6,6,8,65701,Northbound,2023-07-25 00:00:00+00:00,2023-07-25 06:40:58+00:00
45829,6570044906020,06:41:31,06:41:31,5969,100,Waveland/Broadway,0,72974,6,6,8,65701,Northbound,2023-07-25 00:00:00+00:00,2023-07-25 06:41:31+00:00
45831,6570044906020,06:42:03,06:42:03,5970,101,Waveland/Broadway,0,73636,6,6,8,65701,Northbound,2023-07-25 00:00:00+00:00,2023-07-25 06:42:03+00:00
45833,6570044906020,06:42:39,06:42:39,17390,102,Waveland/Broadway,0,74453,6,6,8,65701,Northbound,2023-07-25 00:00:00+00:00,2023-07-25 06:42:39+00:00


In [18]:
active_service_times = get_active_service_times(stop_details, stop_id, direction)
active_service_times

,start_time,end_time
0,2023-07-25 01:23:38+00:00,2023-07-25 01:46:38+00:00
0,2023-07-25 05:07:08+00:00,2023-07-26 01:00:38+00:00


## Actual headway details

In [19]:
halsted_5864_headway_details = get_actual_headways(vehicles, route_id, stop_id, direction, active_service_times)


/Users/kristenhahn/repos/cta_bus_tracker_exploration/headways.py:794: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stop_direction_servicetime['est_headway'].iloc[0] = None


In [20]:
halsted_5864_headway_details

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,...,data_date,end_time,end_pdist,start_time,start_pdist,stpid,stop_pdist,rtdir,est_stop_time,est_headway
10400,1239,2023-07-25 05:47:00+00:00,41.750793,-87.644150,181,9368,8,79th,74427,False,...,2023-07-25,2023-07-25 05:47:00+00:00,74427,2023-07-25 05:42:00+00:00,66646,5864,74427,Southbound,2023-07-25 05:47:00+00:00,NaT
24058,7950,2023-07-25 07:27:00+00:00,41.750610,-87.644133,175,9368,8,79th,74493,False,...,2023-07-25,2023-07-25 07:27:00+00:00,74493,2023-07-25 07:22:00+00:00,68293,5864,74427,Southbound,2023-07-25 07:26:57+00:00,0 days 01:39:57
44691,7990,2023-07-25 09:17:00+00:00,41.749827,-87.644018,138,9368,8,79th,74781,False,...,2023-07-25,2023-07-25 09:17:00+00:00,74781,2023-07-25 09:12:00+00:00,69593,5864,74427,Southbound,2023-07-25 09:16:40+00:00,0 days 01:49:43
47097,7918,2023-07-25 09:32:00+00:00,41.750626,-87.644136,177,9368,8,79th,74487,False,...,2023-07-25,2023-07-25 09:32:00+00:00,74487,2023-07-25 09:27:00+00:00,67167,5864,74427,Southbound,2023-07-25 09:31:58+00:00,0 days 00:15:18
54878,1211,2023-07-25 10:27:00+00:00,41.750547,-87.644126,174,9368,8,79th,74516,False,...,2023-07-25,2023-07-25 10:27:00+00:00,74516,2023-07-25 10:22:00+00:00,67850,5864,74427,Southbound,2023-07-25 10:26:56+00:00,0 days 00:54:58
68343,1586,2023-07-25 12:12:00+00:00,41.749835,-87.644023,144,9368,8,79th,74777,False,...,2023-07-25,2023-07-25 12:12:00+00:00,74777,2023-07-25 12:07:00+00:00,70273,5864,74427,Southbound,2023-07-25 12:11:37+00:00,0 days 01:44:41
78223,1307,2023-07-25 13:27:00+00:00,41.750596,-87.644132,176,9368,8,79th,74498,False,...,2023-07-25,2023-07-25 13:27:00+00:00,74498,2023-07-25 13:22:00+00:00,70767,5864,74427,Southbound,2023-07-25 13:26:54+00:00,0 days 01:15:17
88783,1203,2023-07-25 14:37:00+00:00,41.749787,-87.643908,85,9368,8,79th,74815,False,...,2023-07-25,2023-07-25 14:37:00+00:00,74815,2023-07-25 14:32:00+00:00,70271,5864,74427,Southbound,2023-07-25 14:36:34+00:00,0 days 01:09:40
95907,7963,2023-07-25 15:17:00+00:00,41.749828,-87.644020,151,9368,8,79th,74781,False,...,2023-07-25,2023-07-25 15:17:00+00:00,74781,2023-07-25 15:12:00+00:00,71061,5864,74427,Southbound,2023-07-25 15:16:31+00:00,0 days 00:39:57
102752,1586,2023-07-25 15:52:00+00:00,41.750640,-87.644137,178,9368,8,79th,74482,False,...,2023-07-25,2023-07-25 15:52:00+00:00,74482,2023-07-25 15:47:00+00:00,69154,5864,74427,Southbound,2023-07-25 15:51:57+00:00,0 days 00:35:26


## Scheduled Headway Details

In [21]:
halsted_5864_scheduled_headway_details = get_scheduled_headways(stop_details, stop_id, direction, active_service_times)


In [22]:
halsted_5864_scheduled_headway_details

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,arrival_hour,departure_hour,route_id,service_id,direction,raw_date,stop_time,previous_stop_time,headway
13605,6570015201020,01:23:38,01:23:38,5864,105,79th,0,74437,1,1,8,65712,Southbound,2023-07-25 00:00:00+00:00,2023-07-25 01:23:38+00:00,NaT,NaT
9046,6570009383020,01:46:38,01:46:38,5864,105,79th,0,74437,1,1,8,65712,Southbound,2023-07-25 00:00:00+00:00,2023-07-25 01:46:38+00:00,2023-07-25 01:23:38+00:00,0 days 00:23:00
14771,6570015867020,05:07:08,05:07:08,5864,105,79th,0,74437,5,5,8,65701,Southbound,2023-07-25 00:00:00+00:00,2023-07-25 05:07:08+00:00,NaT,NaT
42682,6570041247020,05:27:08,05:27:08,5864,105,79th,0,74437,5,5,8,65701,Southbound,2023-07-25 00:00:00+00:00,2023-07-25 05:27:08+00:00,2023-07-25 05:07:08+00:00,0 days 00:20:00
28929,6570027695020,05:47:38,05:47:38,5864,105,79th,0,74437,5,5,8,65701,Southbound,2023-07-25 00:00:00+00:00,2023-07-25 05:47:38+00:00,2023-07-25 05:27:08+00:00,0 days 00:20:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2644,6570001884020,23:47:05,23:47:05,5864,105,79th,0,74437,23,23,8,65701,Southbound,2023-07-25 00:00:00+00:00,2023-07-25 23:47:05+00:00,2023-07-25 23:30:35+00:00,0 days 00:16:30
21312,6570021293020,24:05:05,24:05:05,5864,105,79th,0,74437,0,0,8,65701,Southbound,2023-07-25 00:00:00+00:00,2023-07-26 00:05:05+00:00,2023-07-25 23:47:05+00:00,0 days 00:18:00
29141,6570028510020,24:22:35,24:22:35,5864,105,79th,0,74437,0,0,8,65701,Southbound,2023-07-25 00:00:00+00:00,2023-07-26 00:22:35+00:00,2023-07-26 00:05:05+00:00,0 days 00:17:30
1027,6570000414020,24:39:38,24:39:38,5864,105,79th,0,74437,0,0,8,65701,Southbound,2023-07-25 00:00:00+00:00,2023-07-26 00:39:38+00:00,2023-07-26 00:22:35+00:00,0 days 00:17:03
